#  Train a GPT-2 Model on Tweets

by [Max Woolf](http://minimaxir.com)

To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


## GPU

Colaboratory uses a Nvidia T4 GPU, Nvidia K80 GPU, or a P100 GPU. The P100 is ideal. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [1]:
import gpt_2_simple as gpt2
from datetime import datetime

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.

Using the `774M` or `1558M` models is not recommended with tweet data, as tweets are short.

You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [2]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 1.09Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 1.44Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 1.02Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [06:28, 3.65Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 733Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.45Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.78Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "revertetweets.csv"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=2000,
              restore_from='fresh',
              run_name='run2',
              print_every=100,
              sample_every=500,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 14.66it/s]


dataset has 1303303 tokens
Training...
[100 | 314.11] loss=2.38 avg=2.38
[200 | 619.11] loss=2.59 avg=2.49
[300 | 924.23] loss=2.46 avg=2.48
[400 | 1229.23] loss=2.59 avg=2.51
[500 | 1534.74] loss=2.49 avg=2.50
Saving checkpoint/run2/model-500
======== SAMPLE 1 ========
||startoftext|>Hay nuevo más técnicamente que ver a bajillo. ¿Por qué habla? El bar que hace tiempo del cine es que no voy en la vida la pasa es. Un saludo.<|endoftext|>
<|startoftext|>No creo que siempre viene. Un saludo.<|endoftext|>
<|startoftext|>¿Por qué me quedo de la baja? Hablaba de ser lo deseo a ver a mí. Quiero decir que no se la paga cuenta un abrazo. Pero táctica, pero más día a quienes no está muy bien dónde suñado de "que vaya a otro" una guerra de toda lucha que viene paz.<|endoftext|>
<|startoftext|>Un asunto de él. Cada vez todo aquel gilipollez, pero cinco veces hace tiempo. Un saludo.<|endoftext|>
<|startoftext|>No lo sé cómo viene eso, lo lamento que te eso. Hablando debería poner eso a ver cómo el 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run2')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [5]:
gpt2.copy_checkpoint_from_gdrive(run_name='run2')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [3]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run2')

2021-10-03 19:52:59.849256: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-10-03 19:52:59.876492: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3700115000 Hz
2021-10-03 19:52:59.876932: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555c17d1c170 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-03 19:52:59.876946: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-03 19:52:59.882666: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-03 19:52:59.996882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-03 19:52:59.997383: 

Loading checkpoint checkpoint/run2/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-2000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

However, for tweets, generating from the base model alone won't be as fruitful due to the necessary encoding. For a solution to that, see the next cell.

In [4]:
gpt2.generate(sess, run_name='run2')

Perdónero de Alatriste, el mensaje invito.<|endoftext|>
<|startoftext|>Pero "El chico de la di", "El dios de la lluvia llora" y "Un asunto de honor" se me fue una broma "pierda historia".<|endoftext|>
<|startoftext|>Me parece que lo dejo.<|endoftext|>
<|startoftext|>En eso sí. Hay que libros y libros para escribir. Otro abrazo.<|endoftext|>
<|startoftext|>No lo he leído. Un abrazo, amiga mía.<|endoftext|>
<|startoftext|>Existe como los libros. Y de la vida lo hiciera. Pero nunca se sabe.<|endoftext|>
<|startoftext|>No tengo nada en absoluto.<|endoftext|>
<|startoftext|>Lo más correcto es que por ahora "El milagrotón de la sepulcra de las Marcejas".<|endoftext|>
<|startoftext|>Me temo que el término es aburrido.<|endoftext|>
<|startoftext|>Vale. En siempre.<|endoftext|>
<|startoftext|>Me temo que el teatro gay es perfecto. Gracias por el interés, y un saludo.<|endoftext|>
<|startoftext|>Pues cuéntemelo, amiga mía. Un saludo.<|endoftext|>
<|startoftext|>Las dos. Y puedo que lo diga. La m

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

Tweets require specific `prefix`, `truncate`, and `include_prefix` as demoed below.

In [11]:
gpt2.generate(sess,
              length=200,
              temperature=0.7,
              prefix='España',
              truncate='<|endoftext|>',
              include_prefix=True,
              nsamples=20,
              batch_size=20
              )

España y el rey de la ciudadana me parece una miseria despuésica.
España, me parece. Y me parece una copa con los amigos.
España" aprovechando de la primera parte de las que llamen no llevaban tres días, sin duda. En el mar y en las que pasan los días en tres pueblos.
España de "hombre que espontase de lo que nos hacen" es España.
España: cerca la palabra "marca por el mar" y el mundo. Eso lo habría sido y la experiencia. No me diga.
España. Pero no sé qué decirle. Un saludo.
España, español. Y español conocida.
España" y "maristas" (saldrá en lo de desesperación y lo de desesperación,dijo está en el mar y en lo de hacerse o en lo del canto).En España, parece una agresión personal de espantos.
España del XVII, la de Gandía y el XVIII, la de la Reina del Sur, la de Tintín de México y de la XIX. Y ya me contará.
España de la RAE. Un saludo.
España, en el Limón, queda la sepia. Es lo que hay que salir en el mar. Un saludo.
España, algunos de la lengua española, mejillan los míos.
España y

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=200,
                      temperature=1.0,
                      top_p=0.9,
                      prefix='<|startoftext|>',
                      truncate='<|endoftext|>',
                      include_prefix=False,
                      nsamples=1000,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

If you want to generate a large number of 1,000 tweet files without having to click a cell button multiple times, you can use the cell below. You can then download the generated files by right-clicking them in the Files sidebar.

In [ ]:
num_files = 10

for _ in range(num_files):
  gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

  gpt2.generate_to_file(sess,
                        destination_path=gen_file,
                        length=200,
                        temperature=1.0,
                        top_p=0.9,
                        prefix='<|startoftext|>',
                        truncate='<|endoftext|>',
                        include_prefix=False,
                        nsamples=1000,
                        batch_size=20
                        )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019-2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.